In [13]:
import pandas as pd
import numpy as np

In [2]:
from pycaret.regression import *

In [3]:
import mlflow

In [4]:
# mlflow.set_tracking_uri('http://127.0.0.1:5000')

In [5]:
mlflow.set_tracking_uri('http://ec2-54-165-160-184.compute-1.amazonaws.com:5000')

In [6]:
# No incluir columnas
exclude = ['Posted On', 'Area Locality','Floor']

In [7]:
data = pd.read_csv('raw.csv')

In [8]:
data

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [14]:
data.columns

Index(['Posted On', 'BHK', 'Rent', 'Size', 'Floor', 'Area Type',
       'Area Locality', 'City', 'Furnishing Status', 'Tenant Preferred',
       'Bathroom', 'Point of Contact'],
      dtype='object')

In [9]:
s = setup(data, target='Rent', session_id=123, ignore_features=exclude,
          experiment_name='model_rent_lgbm', log_experiment=True)

,Description,Value
0,Session id,123
1,Target,Rent
2,Target type,Regression
3,Original data shape,"(4746, 12)"
4,Transformed data shape,"(4746, 20)"
5,Transformed train set shape,"(3322, 20)"
6,Transformed test set shape,"(1424, 20)"
7,Ignore features,3
8,Ordinal features,1
9,Numeric features,3


In [10]:
# best = compare_models()

In [11]:
best = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17536.6768,3589695689.9159,59914.0692,0.4960,0.4777,0.4394
1,21944.3106,35452591929.5244,188288.5868,0.0879,0.4794,0.3917
2,11215.6283,1041215572.9045,32267.8721,0.4856,0.3927,0.3347
3,15086.9365,2046069742.2467,45233.5024,0.2443,0.4718,0.4191
4,12662.0335,903081264.9332,30051.3105,0.7476,0.4368,0.4154
5,15208.5399,1209334867.8649,34775.4923,0.7407,0.4584,0.4109
6,13174.8815,1512366726.1563,38889.1595,0.4751,0.4684,0.3886
7,10662.7201,715649633.7334,26751.6286,0.6430,0.4047,0.3439
8,13112.9483,2230982034.8064,47233.2725,0.3742,0.4537,0.3560


In [12]:
tuned_model = tune_model(best, n_iter=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,17151.1262,3544750890.3677,59537.8106,0.5023,0.5576,0.4380
1,23019.7573,35356443031.6662,188033.0903,0.0904,0.5258,0.4432
2,9964.6935,443483401.5312,21059.0456,0.7809,0.3861,0.3239
3,12811.9146,941325046.6878,30681.0210,0.6523,0.4457,0.3868
4,11880.4413,867085076.8055,29446.3084,0.7577,0.4618,0.4016
5,14968.1714,1318724860.2891,36314.2515,0.7173,0.4525,0.3823
6,13206.3934,1244761618.9319,35281.1794,0.5680,0.4703,0.4093
7,10470.8634,502313590.6077,22412.3535,0.7494,0.4297,0.3696
8,11754.2375,1532237411.1117,39143.8042,0.5702,0.4549,0.3496


Fitting 10 folds for each of 20 candidates, totalling 200 fits
[LightGBM] [Warning] feature_fraction is set=1.0, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=1.0
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


In [12]:
final = finalize_model(tuned_model)

In [13]:
final

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['BHK', 'Size', 'Bathroom'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Area Type', 'City',
                                             'Furnishing Status',
                                             'Tenant Preferred',
                                             'Point of Contact'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 Tra...
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 LGBMRegressor(bagging_fraction=0.6, bagging_freq=2,
                               feature_fraction=1.0, learning_rate=0.15,
                               min_child_samples=61, min_split_gain=0.8,
                               n_estimators=90, n_jobs=-1, num_leaves=70,
                               random_state=123, reg_alpha=0.0001,
                               reg_lambda=0.05))])

In [14]:
save_model(final,'model_lgb')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['BHK', 'Size', 'Bathroom'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Area Type', 'City',
                                              'Furnishing Status',
                                              'Tenant Preferred',
                                              'Point of Contact'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  Tra...
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_e

In [ ]:
https://8sn7ae77ld.execute-api.us-east-1.amazonaws.com/dev

In [5]:
import requests
import json

url = 'https://8sn7ae77ld.execute-api.us-east-1.amazonaws.com/dev/predict'

data = {
  "BHK": 2,
  "Size": 2000,
  "Area_Type": "Carpet Area",
  "City": "Kolkata",
  "Furnishing_Status": "Unfurnished",
  "Tenant_Preferred": "Bachelors/Family",
  "Bathroom": 2,
  "Point_of_Contact": "Contact Owner"
}


In [6]:
response = requests.post(url, data=json.dumps(data))

In [7]:
response.json()

{'prediction': 38202.46008740821}